In [ ]:
initial_path = 'peptide-QML/'
# initial_path = '../'

In [ ]:
import numpy as np
import sys
sys.path.append(initial_path)

%load_ext autoreload
%autoreload 2
from my_code import functions as f
from my_code import pytorch_model as ptm
from my_code import quantum_nodes as qn
from my_code import pytorch_wrappers as pw

In [ ]:
for num in [4, 6, 8, 10, 12]:
    print("## --- Num: ", num, " --- ##")
    print("")
    print("")

    for type_energy in ['', '_Single', '_Pair']:

        # -- DATA -- #

        # name of the notebook
        day = "0901"
        name_notebook = "{}-{}c_em_1qm{}_v2.ipynb".format(day, num, type_energy)
        # save the name of the notebook
        with open('{}_notebooks_names_classical.txt'.format(day), 'a') as file:
            file.write(name_notebook + '\n')

        # load data
        data_file_path = '{}data/Scores/PET/generated/bb14_Strings_Energies{}_10_000_{}_aa.txt'.format(initial_path, type_energy, num)  
        string_list, number_list = f.read_data_file(data_file_path)
        score_list = np.array(number_list)/1000
        vector_list = np.array([f.string_to_numbers(string) for string in string_list]) 
        X, Y, X_validation, Y_validation = f.create_validating_set(vector_list, score_list, percentage=0.1)

        # Define the dataset
        input_data = ptm.torch.tensor(X) 
        target_data = ptm.torch.tensor(Y).view(-1, 1)

        # Define the validation set
        input_validation = ptm.torch.tensor(X_validation)
        target_validation = ptm.torch.tensor(Y_validation).view(-1, 1)


        # -- TRAINING LOOP -- #
        for iii in range(5):

            for block_n_layers in range(1, 21):
                print(iii, " # ------- block_n_layers: ", block_n_layers, " ------- #")
                print("")
                print("")

                # -- CLASSICAL MODEL -- #
                n_aminoacids = len(string_list[0])
                input_dim = n_aminoacids

                import torch.nn as nn
                import torch

                class Reshape(nn.Module):
                    def __init__(self):
                        super(Reshape, self).__init__()

                    def forward(self, x):
                        dims = len(x.shape)
                        return torch.transpose(x, dims-2, dims-1).reshape(x.shape[-3] if dims==3 else 1, x.shape[-2]*x.shape[-1]).squeeze()

                layers =  [ptm.nn.Embedding(num_embeddings=19, embedding_dim=1), Reshape()]
                # for i in range(block_n_layers//2):
                #     layers += [ptm.nn.Linear(input_dim*(i+1), input_dim*(i+2)), ptm.nn.Sigmoid()]
                # for i in range(block_n_layers//2, 0, -1):
                #     layers += [ptm.nn.Linear(input_dim*(i+1), input_dim*(i)), ptm.nn.Sigmoid()]   
                for i in range(block_n_layers):
                    layers += [ptm.nn.Linear(input_dim, input_dim), ptm.nn.Sigmoid()]      
                layers += [ptm.nn.Linear(input_dim,1), ptm.nn.Sigmoid(), ptm.nn.Linear(1,1)]

                # Create model and set data:
                model = ptm.pytorch_model(
                    layers,    
                    save_options = {'initial_path': initial_path, 'name_notebook': name_notebook},
                )
                model.set_data(
                    data_X=input_data, 
                    data_Y=target_data, 
                    data_X_validation=input_validation,
                    data_Y_validation=target_validation
                )
                model.keep_track_params=False


                # -- TRAINING -- #
                import torch.optim as optim

                # train the model
                model.train(
                    optimizer= optim.Adam,
                    optimizer_options={'lr': 0.02},
                    num_epochs = 100,                
                    batch_size = 32,
                    stop_training_options = {
                        'lookback_epochs': 5, 
                        'threshold_slope': 0.001,
                        'threshold_std_dev': 0.2
                    },
                )

                # -- SAVE -- #

                # dump the object
                model.dump()

                # save string of the model
                model.save_str(
                    metadata={
                        "data_file_path": data_file_path,
                    }
                )

                # plot the losses of the trainig loop and save the figures
                model.plot_losses(save=True, save_txt=True)

                #save all the parameters of the model and the intermediate parameters of the model
                model.save_state_dict(intermediate=False)

                # plot the validation and save it
                model.plot_validation(save=True, fig_size=(9,9), percentatge=1)

                # print validation and save it
                model.str_validation(save=True, precision=4, percentatge=1, printing=False)

In [ ]:
# push changes to git
if initial_path != '../':
    !cd peptide-QML && git add . && git commit -m "data trained model" && git push